In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters

import pdfplumber


In [255]:
os.getcwd()
#os.chdir("/Users/veesheen/Desktop")

'/Users/veesheenyuen/code/veeyuen/SAA'

# Extract results from NSG

In [271]:
if 'Disqualified' in fragment:
    fragment.insert(7, ' ')
    
else:
    fragment.insert(len(fragment), ' ')

In [573]:
import re

pattern='[A-Z]{2,5}'

for i in range(len(fragment)):

    if re.search(pattern, fragment[i]):
        print(i, re.search(pattern, fragment[i]))
        index=i  # set index to second value if there are two uppercase values encountered
        print(index)



1 <re.Match object; span=(0, 4), match='CHIJ'>
1
6 <re.Match object; span=(0, 3), match='SNG'>
6


In [384]:
# Regex pattern testing

#pattern='([A-Z][a-z]+\s){2,4}'

#pattern='\d{1,3}{(?<=\s)[A-Z]{3,5}'

#pattern='[A-Z]{2,4}'

pattern='\d\s\[A-Z]{3,}(?=Q|q|[0-9]{2,3})'


#string='8 CHIJ Sec (Toa Payoh) HIJ 2 7 00:54.31'
#string='999 Chestnut Drive Sec CDS 2 3 Withdrawal'
#string='7 CHIJ Katong Convent KC 5 7 00:54.10'
#string = '1 XIN BOWEN 587 HCI Finals 6 40.15 9'


#matches=re.finditer(pattern, string)

matches=re.findall(pattern, string)

print(matches)

#print(type(matches))

#for match in matches:
#    print(match)



[]


In [126]:
# Test splicing and extracting name

#s = "4 PHOEBE ANG KE WEI Q 800 CGS h4 2 01:05.62"
#s="2 Sarah Chong Xin RI 1 12 4.78 7"
#s='7 CHIJ Katong Convent KC 5 7 00:54.10'
#s='4 WANG HANCHEN 584 HCI Finals 1 36.79 5'
#s='1 XIN BOWEN 587 HCI Finals 6 40.15 9'
#s='3 SINGAPORE SPORTS SCHOOL Q 598 SSP h1 3 04:32.84'
s='16 TEMASEK SECONDARY SCHOOL 873 TMS h4 3 05:08.64'

lpos = re.search('\s', s)
#rpos = re.search('\b[A-Z]{2-4}\b\d', s)

#rpos = re.search('\sQ\s', s)
#rpos = re.search('\s[A-Z]{2,4}?', s)
#rpos = re.search('\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?|\s[A-Z]{2,4}\s\d?', s)
rpos = re.search('\d\s.{5,40}?\s(Q|q|\d)', s)



split_pos_end=rpos.end()-1
split_pos_start=lpos.start()+1



l = s[:lpos.start()]
r = s[rpos.start():]

#name=s[lpos.start():rpos.start()]

name=s[split_pos_start:split_pos_end]

print(name)

TEMASEK SECONDARY SCHOOL 


In [87]:
# test regex for event name extraction

s='Event: S13-09 Discus - B DIVISION BOYS Finals'
event_pattern=':\s.{5,20}\s\-'


re.search(event_pattern, s)


<re.Match object; span=(5, 22), match=': S13-09 Discus -'>

In [38]:
# Test pdfplumber

#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/sectrack_audit_2010_1-1')
#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/')
#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6_1-1/') 
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/') 


import pdfplumber

#file="sectrack_audit_2010_211-211.pdf"
#file="Session 13_13-end.pdf"
#file="Session 13_1-4.pdf"
#file="sectrack_audit_2010_28-28.pdf"
#file="Session 13_5-6.pdf"
#file='Session 13_5-6_2-end.pdf'   # stranded pdf
file='SGO24.pdf'


#table_settings = {
#    "vertical_strategy": "text",
#    "horizontal_strategy": "text",
#    }


with pdfplumber.open(file) as pdf:
    
    page = pdf.pages[0]
    table=page.extract_table()
    text=page.extract_text()


splitted=text.splitlines()

splitted

["Singapore Athletics Assoc.-Open Champ - Organization License Hy-Tek's MEET MANAGER 9:03 PM 19 Apr 2024 Page 1",
 '84th Singapore Open Track & Field - 18 Apr 2024 to 19 Apr 2024',
 'Championships 2024',
 'Singapore Sportshub',
 'Results',
 'Event 101 Women Shot Put Open',
 '=========================================================================',
 'Name Age Team Seed Finals',
 '=========================================================================',
 'Flight 1',
 '1 # 154 Intadis, Areerat 28 Thailand 16.71m 14.66m',
 '13.78m 14.41m X 14.29m 14.34m 14.66m',
 '2 # 84 Yee Wai Teng, Mel 26 Singapore 12.01m 10.60m',
 'X 10.33m X 10.16m 10.60m 10.44m',
 '3 # 172 Neo, Ee Nin 18 Wings Athlet 9.21m 9.32m',
 '8.31m 9.30m 9.17m 8.56m 9.32m 8.59m',
 'Event 102 Women 1500 Meter Run Open',
 '=========================================================================',
 'Name Age Team Seed Finals',
 '=========================================================================',
 'Section 1',
 '1 # 4

In [42]:
print(table)

None


In [210]:
pd.DataFrame(splitted)

,0
0,23 BUKIT VIEW SECONDARY 999 BTV h3 4 05:51.77
1,SCHOOL
2,/


In [35]:
def old_assign(table):

    global index
    
    exclusions=[]
    
    df=pd.DataFrame(table)
    columns=df.iloc[6,0]
    names=columns.split(' ')

    df2 = pd.DataFrame(columns=names)
    
    new_df = df2.iloc[:, :-2]    # remove last 2 extraneous columns
     
    
    for i in range(7,(len(df)-1)):
            
            
        if not re.search('^\d', table[i]): # skip row if disqualified or withdrawn (does not begin with a digit)

            
            continue

            
        s=table[i]
        
        lpos = re.search('\s', s)
        #rpos = re.search('\sQ\s', s)
        #rpos = re.search('\s[A-Z]{2,4}', s)
        
        matches=re.findall('\s[A-Z]{2,4}', s)  # find how many uppercase shortforms for school
        
        if len(matches)==2:
            rpos=re.search(matches[1], s)
        else:
            rpos = re.search('\s[A-Z]{2,4}', s)

            
        # Splice string to extract name of competitor

            
        l = s[:lpos.start()] # left string post splicing
        r = s[rpos.start():] # right string post slicing
         
        name=s[lpos.start():rpos.start()] # extract competitor name
                
        list=r.split(' ')        

        combined=l+ ',' + name + ', '.join(list)

        row=combined.split(",")
                        
        if len(row)!=6:
            
            n=len(row)-6
            row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns                 
                    
        new_df.loc[len(new_df)] = row

    return new_df

In [36]:
old_assign(splitted)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/484862590.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/484862590.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/484862590.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,Pos,Competitor,Team,Ht,Ln,Result
0,1,Shawn Poh Zhi Kang,SKS,13,7,00:24.20
1,2,Liaw Jian Hui,AMK,3,7,00:24.68
2,3,Zubin Percy Muncherji,ACS(BR),8,7,00:24.77
3,4,Terence Loh Junheng,ACS(I),2,4,00:24.84
4,5,Md Nur Syahirran,SSP,10,3,00:24.90
5,6,Ong Sin Yao,CCH,2,7,00:24.97
6,7,Wu Chee Keen Daniel,DSS,9,1,00:25.05
7,8,Jonathan Lew Lian Xin,RI,11,5,00:25.06
8,9,Ang Li Sheng,AND,2,6,00:25.16
9,10,Teo Yu Ming,RI,6,6,00:25.62


In [73]:
# Old version- does not capture REMARKS and NR columns

def new_format_parser(row_index, names, table, master_df):
            
    # Define regex patterns
    
    new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'
    
    stranded = "no"  # flag for stranded pdf pages

        
    # Put table that has been processed into a dataframe
    
    df=pd.DataFrame(table)
            
    columns=[]
    
    
    # Initialize new df and remove unnecessary columns
    
    if row_index!=0 and names is not None: #check that its coming from not a stranded pdf with no header
            
        df2 = pd.DataFrame(columns=names)        
    
        new_df = df2.iloc[:, :-2]    # remove 'Remarks' and 'NR' columns
    
    else:
        
        print('reached this first point for a stranded pdf')
        
        stranded="yes"
        
        new_df = master_df.iloc[:0] # new_df columns to come from the master df
        
        row_index=-1  # move back row_index for a stranded pdf
        
    
#    for i in range(row_index+1,(len(df)-1)):  # iterate over df - only stranded pdf should start from zero. Previous code - revert to original  


#  check for stranded pdfs with stranded names

    for i in range(row_index+1,(len(df))):  # iterate over df - only stranded pdf should start from zero        

        
        if 'DNM' in table[i]:
                    
            continue # skip row if DNM, DQ, DNF etc.
                                    
        if not re.search('^\d', table[i]): # look for stranded parts of names (does not begin with a digit). But need to test for standed names in stranded pdf
        
        
            try:  # remove try block to revert back to previous working state
                
                row[1]=row[1]+table[i] # retrieve previous row and add back stranded string to name. Stranded pdf will throw and error here
                
                new_df.drop(new_df.tail(1).index,inplace=True) # drop last row with incomplete name
                                        
                new_df.loc[len(new_df)] = row  # add back amended row
                
                print(row)
                                    
                continue  # then skip to next row
            
            except:
                
                print('reached this second point for a stranded pdf')
                
                pass                 # move on if it is actually a stranded pdf
                              
                
        string=table[i] 
        
        print(string)
                                
        lpos = re.search('\s', string)
        rpos = re.search(new_pattern, string)
                                            
            
        # Splice string to extract name of competitor

        split_pos_end=rpos.end()-1      # adjust the splicing position to only capture the name
        split_pos_start=lpos.start()+1

            
        left_string = string[:lpos.start()] # left string post splicing
        right_string = string[split_pos_end:] # right string post slicing
                         
        name=string[split_pos_start:split_pos_end]  # extract competitor name
                                        
        list=right_string.split(' ')        # put everything coming after the name into a list   
                
                
        if len(list)==6:    # check length of list. If less than 7, append blank fields for Pos and W/G 
                    
            list.extend([' ', ' '])
                    
        elif len(list)==5:
                    
            list.extend([' ', ' '])
                                                            
                
        combined=left_string+ ';' + name +'; '+ '; '.join(list)  # use ';' in case names have a ','

        row=combined.split(';')        
                                                                             

        # check if Q or q between tag and name. If not, fill in with empty space.
        # don't overwrite master df if there is a stranded pdf
                
        if row[2].strip()=='Q' or row[2].strip()=='q' or row[2] is None :
                    
            new_df.loc[len(new_df)] = row
                        
#            master_df=new_df      # previous

        else:
                    
            row.insert(2, ' ')  # insert a blank field if no Q or q    
            
            print(row)
                
            new_df.loc[len(new_df)] = row
            
            
# QAed to this point for a stranded pdf. Need to figure out how to attach stranded row to master df
            
            
#            master_df=new_df      # previous   

    
    return new_df

In [97]:
def extraction_old(table, master_df):  # old version without calling new_format_parser function

    global index
        
        
    # Regex patterns to extract competitor names, event name etc.
    
    #new_pattern='\d\s.{5,20}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='\d\s[A-Z\s\,]{5,30}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='^[A-Z\s]*$'
    
    new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'

    
    # Initialize new df and extract event information
    
    df=pd.DataFrame(table)
    
    columns=[]
    
    
    for index, row in df.iterrows(): # find row containing event details and column names
        
        
        if 'Event' in row[0]: # data is in column '0' 
            
            event=row[0]   # set string for extraction of event info
            
                        
            epos = re.search(event_pattern, event)
                        
            split_end=epos.end()-1      # adjust the splicing position to only capture the event name
            split_start=epos.start()+1

            event=event[split_start:split_end]
                                    
            
            if 'BOYS' in event or 'Boys' in event:
                gender='Male'
                
            if 'GIRLS' in event or 'Girls' in event:
                gender='Female'    
                
            
            if 'Finals' in event:
                round='Finals'
            
            region='Local'  
            
       
            
        if 'Competitor' in row[0]: # extract column names
            
            row_index=index
            columns=df.iloc[row_index,0]
            names=columns.split(' ')
            
    
    
    
    # Check pdf format and begin extraction of results
    
    if 'W/G' not in columns:  # if older pdf format without a 'W/G column'
            
        df2 = pd.DataFrame(columns=names)        
    
        new_df = df2.iloc[:, :-2]    # remove Remarks and N/R columns
         
        for i in range(row_index+1,(len(df)-1)):                    
            
            if not re.search('^\d', table[i]): # skip row if disqualified or withdrawn (does not begin with a digit)
            
                continue
       
            string=table[i]
                                
            lpos = re.search('\s', string)
                #rpos = re.search('\sQ\s', s)
                #rpos = re.search('\s[A-Z]{2,4}', s)
            
            matches=re.findall(school_pattern, string)  # find how many uppercase shortforms for school
                                
            if len(matches)==2:
                
                rpos=re.search(matches[1], string) # choose second match if there are two 
                    
            else:
                    
                rpos = re.search(old_pattern, string)
                
            
        # Splice string to extract name of competitor

            
            l = string[:lpos.start()] # left string post splicing
            r = string[rpos.start():] # right string post slicing
         
            name=string[lpos.start():rpos.start()] # extract competitor name
                                
            list=r.split(' ')   # split the parts after the name    
            
            combined=l+ ',' + name + ', '.join(list)

            row=combined.split(",")
                        
                        
            if ('W/G' not in names) and len(row)!=6:  # if W/G not in list of columns
            
                n=len(row)-6
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
            
            elif ('W/G' in names) and len(row)>8:
            
                n=len(row)-8
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
                                        
            new_df.loc[len(new_df)] = row
            
        return new_df
            
    elif 'W/G' in columns: # new format 
            
            df2 = pd.DataFrame(columns=names)        
    
            new_df = df2.iloc[:, :-2]    # remove 'Remarks' and 'NR' columns        

         
            for i in range(row_index+1,(len(df)-1)):
                                
                if 'DNM' in table[i]:
                    
                    continue # skip row if DNM, DQ, DNF etc.
                            
                if not re.search('^\d', table[i]): # look for stranded parts of names (does not begin with a digit)
            
                    row[1]=row[1]+table[i] # retrieve previous row and add back stranded string to name
                
                    new_df.drop(new_df.tail(1).index,inplace=True) # drop last row with incomplete name
                                        
                    new_df.loc[len(new_df)] = row  # add back amended row
                    
                    continue  # then skip to next row
                              

                string=table[i]
                                
                lpos = re.search('\s', string)
                rpos = re.search(new_pattern, string)
                                            
            
        # Splice string to extract name of competitor

                split_pos_end=rpos.end()-1      # adjust the splicing position to only capture the name
                split_pos_start=lpos.start()+1

            
                left_string = string[:lpos.start()] # left string post splicing
                right_string = string[split_pos_end:] # right string post slicing
                         
                name=string[split_pos_start:split_pos_end]  # extract competitor name            
                                
                list=right_string.split(' ')        # put everything coming after the name into a list   
                
                
                if len(list)==6:    # check length of list. If less than 7, append blank fields for Pos and W/G 
                    
                    list.extend([' ', ' '])
                    
                elif len(list)==5:
                    
                    list.extend([' ', ' '])
                                                            
                
                combined=left_string+ ';' + name +'; '+ '; '.join(list)  # use ';' in case names have a ','

                row=combined.split(';')
                                                                                
                
        # check if Q or q between tag and name. If not, fill in with empty space.
        
                
                if row[2].strip()=='Q' or row[2] is None:
                    
                    pass
                    
                else:
                    
                    row.insert(2, ' ')  # insert a blank field if no Q or q
                    
                                            
                new_df.loc[len(new_df)] = row
                
                print(row)
                    
                    
            return new_df

In [40]:
master_df=None

master_df=extraction_old(splitted, master_df)  # using old extraction function

0 Singapore Schools Sports Council 14-04-2010
1 51st National Inter-School Track and Field Championships 2010 10:02 PM
2 Audit List Page 28
3 Event : 200 Metres (C-Boys) Type : All Heats
4 02-03
5 Record : Mohd Yusof Bin Azhari SJI 2003 ET 00:22.99s
6 Pos Competitor Team Ht Ln Result Pts Remark
7 1 Shawn Poh Zhi Kang SKS 13 7 00:24.20
8 2 Liaw Jian Hui AMK 3 7 00:24.68
9 3 Zubin Percy Muncherji ACS(BR) 8 7 00:24.77
10 4 Terence Loh Junheng ACS(I) 2 4 00:24.84
11 5 Md Nur Syahirran SSP 10 3 00:24.90
12 6 Ong Sin Yao CCH 2 7 00:24.97
13 7 Wu Chee Keen Daniel DSS 9 1 00:25.05
14 8 Jonathan Lew Lian Xin RI 11 5 00:25.06
15 9 Ang Li Sheng AND 2 6 00:25.16
16 10 Teo Yu Ming RI 6 6 00:25.62
17 11 Clyde Tan Jin Shuen ACS(I) 8 6 00:25.68
18 12 Chua Cheng Ying SJI 13 6 00:25.69
19 13 Puah Li An NTS 4 5 00:25.70
20 14 Raymond Scott Lee Chian Hoong SJI 2 5 00:25.84
21 15 Ridwan Bin Yahya TMS 1 8 00:25.85
22 16 Tan Hui Ming Zac NSS 5 8 00:25.89
23 17 Daryl Shane Lee TMS 5 5 00:25.93
24 18 Chin Yew 

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/2665074277.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/2665074277.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_67368/2665074277.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [30]:
master_df

,Pos,Competitor,Team,Ht,Ln,Result
0,1,Singapore Sports School,SSP,1,4,00:49.67
1,2,CHIJ St Nicholas Girls' School,SNG,1,5,00:51.41
2,3,Nanyang Girls' High School,NYGH,1,6,00:52.25
3,4,Cedar Girls' Sec School,CG,1,3,00:52.90
4,5,CHIJ Sec (Toa Payoh),HIJ,1,7,00:53.88
5,6,CHIJ Katong Convent,KC,1,2,00:55.58
6,7,North Vista Sec School,NV,1,8,00:55.69
7,8,Raffles Girls' School (Sec),RGS,1,1,00:57.27


In [341]:
# Extract list of primary and secondary schools
# Using BS4

URL ='https://en.wikipedia.org/wiki/List_of_secondary_schools_in_Singapore'
page=requests.get(URL)

soup=BeautifulSoup(page.content, 'html.parser')


In [342]:
# Creating list with all tables
tables = soup.find_all('table')

#  Looking for the table with the classes 'wikitable' and 'sortable'
table = soup.find('table', class_='wikitable')

In [343]:
for a in soup.find_all('table'):
    print(a.text)




Name
Type
SchoolCode
Area[4]
Notes / Affiliations (Admissions / Academic Programmes)
Website


Admiralty Secondary School
Government
3072
Woodlands

[1]


Ahmad Ibrahim Secondary School
Government
3201
Yishun

[2]


Anderson Secondary School
GovernmentAutonomous
3001
Ang Mo Kio

[3]


Anglican High School
Government-aidedAutonomousSAP

7101
Bedok

[4]


Anglo-Chinese School (Barker Road)
Government-aided
7032
Newton
Affiliated to:
Methodist Girls' School (Secondary)

[5]


Anglo-Chinese School (Independent)
IndependentIP
IP:9161Express:7001

Queenstown

Offers the International Baccalaureate certificate
Affiliated to:
Methodist Girls' School (Secondary) (IP)

[6]


Ang Mo Kio Secondary School
Government
3026
Ang Mo Kio

[7]


Assumption English School
Government-aided
7002
Bukit Panjang

Affiliated to:
Catholic Junior College (Non-IP)[5]

[8]


Bartley Secondary School
Government
3002
Toa Payoh

[9]


Beatty Secondary School
Government
3003
Toa Payoh

[10]


Bedok Green Secondary Sch

In [364]:
# Extract list of schools using Pandas read_html

sschool_html = pd.read_html("List of secondary schools in Singapore - Wikipedia.html")
pschool_html = pd.read_html("List of primary schools in Singapore - Wikipedia.html")

In [357]:
sschool_html[0]['Name']

0              Admiralty Secondary School
1          Ahmad Ibrahim Secondary School
2               Anderson Secondary School
3                    Anglican High School
4      Anglo-Chinese School (Barker Road)
                      ...                
140                Yuhua Secondary School
141          Yusof Ishak Secondary School
142               Yuying Secondary School
143             Zhenghua Secondary School
144             Zhonghua Secondary School
Name: Name, Length: 145, dtype: object

In [376]:
sec_schools=sschool_html[0]['Name'].tolist()

In [377]:
sec_schools

['Admiralty Secondary School',
 'Ahmad Ibrahim Secondary School',
 'Anderson Secondary School',
 'Anglican High School',
 'Anglo-Chinese School (Barker Road)',
 'Anglo-Chinese School (Independent)',
 'Ang Mo Kio Secondary School',
 'Assumption English School',
 'Bartley Secondary School',
 'Beatty Secondary School',
 'Bedok Green Secondary School',
 'Bedok South Secondary School',
 'Bedok View Secondary School',
 'Bendemeer Secondary School',
 'Boon Lay Secondary School',
 'Bowen Secondary School',
 'Broadrick Secondary School',
 'Bukit Batok Secondary School',
 'Bukit Merah Secondary School',
 'Bukit Panjang Government High School',
 'Bukit View Secondary School',
 'Catholic High School',
 'Canberra Secondary School',
 "Cedar Girls' Secondary School",
 'Changkat Changi Secondary School',
 'CHIJ Katong Convent (Secondary)',
 'CHIJ Secondary (Toa Payoh)',
 "CHIJ St. Joseph's Convent",
 "CHIJ St. Nicholas Girls' School",
 "CHIJ St. Theresa's Convent",
 'Chua Chu Kang Secondary School',
 

In [367]:
pschool_html[1]

,Name,Funding,Type,Area[3],Notes,Website,School Code
0,Admiralty Primary School,Government,Mixed,Woodlands,NaN,[1],1744
1,Ahmad Ibrahim Primary School,Government,Mixed,Yishun,NaN,[2],1738
2,Ai Tong School,"Government-aided, SAP",Mixed,Bishan,Affiliated to Singapore Hokkien Huay Kuan[4],[3],5625
3,Alexandra Primary School,Government,Mixed,Bukit Merah,NaN,[4],1266
4,Anchor Green Primary School,Government,Mixed,Sengkang,NaN,[5],1254
...,...,...,...,...,...,...,...
180,Yuhua Primary School,Government,Mixed,Jurong East,NaN,[180],1656
181,Yumin Primary School,Government,Mixed,Tampines,NaN,[181],1219
182,Zhangde Primary School,Government,Mixed,Bukit Merah,NaN,[182],1199
183,Zhenghua Primary School,Government,Mixed,Bukit Panjang,NaN,[183],1240


In [374]:
prim_schools=pschool_html[1]['Name'].tolist()

In [375]:
prim_schools

['Admiralty Primary School',
 'Ahmad Ibrahim Primary School',
 'Ai Tong School',
 'Alexandra Primary School',
 'Anchor Green Primary School',
 'Anderson Primary School',
 'Anglo-Chinese School (Junior)',
 'Anglo-Chinese School (Primary)',
 'Angsana Primary School',
 'Ang Mo Kio Primary School',
 'Beacon Primary School',
 'Bedok Green Primary School',
 'Bendemeer Primary School',
 'Blangah Rise Primary School',
 'Boon Lay Garden Primary School',
 'Bukit Panjang Primary School',
 'Bukit Timah Primary School',
 'Bukit View Primary School',
 'Canberra Primary School',
 'Canossa Catholic Primary School',
 'Cantonment Primary School',
 'Casuarina Primary School',
 'Catholic High School (Primary)',
 'Cedar Primary School',
 'Changkat Primary School',
 'CHIJ (Katong) Primary',
 'CHIJ (Kellock)',
 'CHIJ Our Lady of Good Counsel',
 'CHIJ Our Lady of the Nativity',
 'CHIJ Our Lady Queen of Peace',
 'CHIJ Primary (Toa Payoh)',
 "CHIJ St. Nicholas Girls' School (Primary Section)",
 'Chongfu School'

In [378]:
all_schools = prim_schools+sec_schools

In [379]:
all_schools

['Admiralty Primary School',
 'Ahmad Ibrahim Primary School',
 'Ai Tong School',
 'Alexandra Primary School',
 'Anchor Green Primary School',
 'Anderson Primary School',
 'Anglo-Chinese School (Junior)',
 'Anglo-Chinese School (Primary)',
 'Angsana Primary School',
 'Ang Mo Kio Primary School',
 'Beacon Primary School',
 'Bedok Green Primary School',
 'Bendemeer Primary School',
 'Blangah Rise Primary School',
 'Boon Lay Garden Primary School',
 'Bukit Panjang Primary School',
 'Bukit Timah Primary School',
 'Bukit View Primary School',
 'Canberra Primary School',
 'Canossa Catholic Primary School',
 'Cantonment Primary School',
 'Casuarina Primary School',
 'Catholic High School (Primary)',
 'Cedar Primary School',
 'Changkat Primary School',
 'CHIJ (Katong) Primary',
 'CHIJ (Kellock)',
 'CHIJ Our Lady of Good Counsel',
 'CHIJ Our Lady of the Nativity',
 'CHIJ Our Lady Queen of Peace',
 'CHIJ Primary (Toa Payoh)',
 "CHIJ St. Nicholas Girls' School (Primary Section)",
 'Chongfu School'

In [192]:
# Test scan of directory files and opening

directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6_1-1/"
 
# Iterate over files in directory
for filename in os.scandir(directory):
    # Open file
    with open(os.path.join(directory, filename)) as f:
        print(f"Content of '{filename}'")    
    
    with pdfplumber.open(file) as pdf:
    
        page = pdf.pages[0]
        table=page.extract_table()
        text=page.extract_text()


    splitted=text.splitlines()

    splitted

Content of '<DirEntry 'Session 13_5-6_2-end.pdf'>'
Content of '<DirEntry 'Session 13_5-6_1-1.pdf'>'


In [243]:
splitted

['Audited List',
 'Event: S13-03 4 X 400m Relay - B DIVISION GIRLS Heats',
 'Schools National Record Singapore Sports School SSP (ET)04:01.98s 2011',
 'Championships Record Singapore Sports School SSP (ET)04:01.98s 2011',
 'POS Competitor Q Tag Team Heat Ln Result Pts W/G Remarks NR',
 '1 SINGAPORE SPORTS SCHOOL Q 720 SSP h3 3 04:23.41',
 "2 NANYANG GIRLS' HIGH Q 1022 NYGH h3 6 04:24.02",
 'SCHOOL',
 "3 CRESCENT GIRLS' SCHOOL Q 791 CGS h3 2 04:24.09",
 "4 CEDAR GIRLS' SECONDARY Q 871 CG h2 4 04:25.16",
 'SCHOOL',
 "5 CHIJ ST. NICHOLAS GIRLS' Q 656 SNG h2 5 04:30.99",
 'SCHOOL (SECONDARY)',
 '6 NATIONAL JUNIOR COLLEGE Q 810 NJC h2 8 04:32.69',
 '7 DUNMAN HIGH SCHOOL Q 930 DHS h2 2 04:37.82',
 '8 CHIJ SECONDARY (TOA PAYOH) Q 747 HIJ h3 5 04:39.52',
 "9 RAFFLES GIRLS' SCHOOL Q 845 RGS h2 1 04:40.60",
 '(SECONDARY)',
 '10 AHMAD IBRAHIM SECONDARY 648 AIS h1 6 04:43.48',
 'SCHOOL',
 '11 PAYA LEBAR METHODIST 953 PLMGS h3 1 04:48.28',
 "GIRLS' SCHOOL (SECONDARY)",
 '12 NORTH VISTA SECONDARY 70

In [477]:
def sort_directory(directory):
    items = os.listdir(directory)
    sorted_items = sorted(items)
    return sorted_items

In [246]:
sort_directory(directory)

['Session 13_5-6_1-1.pdf', 'Session 13_5-6_2-end.pdf']

In [78]:
# This is the starting point. Need 'extraction' and 'new_format_parser' functons.
# Test scan of directory with multiple files and opening 


#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6_1-1/')

directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/Deconstructed/"
 
# Iterate over files in directory

sorted_items=sort_directory(directory)

master_df=pd.DataFrame()  # initialize empty master df

for file in sorted_items:
    
    print(file)

    with pdfplumber.open(file) as pdf:
    
        page = pdf.pages[0]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()


    splitted=text.splitlines() # if more than one page should append the two 'splitted' files before extraction
            
    master_df=extraction(splitted, master_df)
    
    


S01-01_TO_11_deconstructed.pdf


FileNotFoundError: [Errno 2] No such file or directory: 'S01-01_TO_11_deconstructed.pdf'

In [126]:
splitted

['23 BUKIT VIEW SECONDARY 999 BTV h3 4 05:51.77', 'SCHOOL', '/']

In [271]:
master_df

,POS,Competitor,Q,Tag,Team,Heat,Ln,Result,Pts,W/G
0,1,SINGAPORE SPORTS SCHOOL,Q,720,SSP,h3,3,04:23.41,,
1,2,NANYANG GIRLS' HIGH SCHOOL,Q,1022,NYGH,h3,6,04:24.02,,
2,3,CRESCENT GIRLS' SCHOOL,Q,791,CGS,h3,2,04:24.09,,
3,4,CEDAR GIRLS' SECONDARY SCHOOL,Q,871,CG,h2,4,04:25.16,,
4,5,CHIJ ST. NICHOLAS GIRLS' SCHOOL (SECONDARY),Q,656,SNG,h2,5,04:30.99,,
5,6,NATIONAL JUNIOR COLLEGE,Q,810,NJC,h2,8,04:32.69,,
6,7,DUNMAN HIGH SCHOOL,Q,930,DHS,h2,2,04:37.82,,
7,8,CHIJ SECONDARY (TOA PAYOH),Q,747,HIJ,h3,5,04:39.52,,
8,9,RAFFLES GIRLS' SCHOOL (SECONDARY),Q,845,RGS,h2,1,04:40.60,,
9,10,AHMAD IBRAHIM SECONDARY SCHOOL,,648,AIS,h1,6,04:43.48,,


In [269]:
def extraction(table, master_df):  # second latest version

    global index, row_index
        
        
    # Regex patterns to extract competitor names, event name etc.
    
    #new_pattern='\d\s.{5,20}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='\d\s[A-Z\s\,]{5,30}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='^[A-Z\s]*$'
    
    new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'
    
    event = None
    
    names = None
    
    temp_df = pd.DataFrame()

    
    # Put extracted info from pdf into a df and extract event information
    
    df=pd.DataFrame(table)
    
    columns=[]
    
        
    for index, row in df.iterrows(): # find row containing event details and column names
                
        
        if 'Event' in row[0]:
            
            event=row[0]
                                    
            epos = re.search(event_pattern, event)
                        
            split_end=epos.end()-1      # adjust the splicing position to only capture the event name
            split_start=epos.start()+1

            event=event[split_start:split_end]
                                    
            
            if 'BOYS' in event or 'Boys' in event:
                gender='Male'
                
            if 'GIRLS' in event or 'Girls' in event:
                gender='Female'    
                
            
            if 'Finals' in event:
                round='Finals'
            
            region='Local'          
                  
            
        if 'Competitor' in row[0]: # extract column names
            
            row_index=index
            columns=df.iloc[row_index,0]
            names=columns.split(' ')
            
    
    if (event and names) is None:  # if it is a stranded page (no header) that is cutoff from the main pdf
                                # Change this code to add iteration over pages. Just append splitted 2nd page.
                                
            temp_df = master_df.iloc[:0]  # initiate empty df and copy column names
                                    
            row_index=0   # if no header 
            
            names = None  # retrieve master df and append instead
            
            print('here')
            
            print(table)
                                            
            temp_df=new_format_parser(row_index, names, table, master_df) # need to concat master_df and temp df
            
            print('reached this 3rd point for a stranded pdf')
            
            master_df= pd.concat([master_df, temp_df])  # this concat is not working as temp_df is empty
                    
            return master_df # exit function

            
    
    # Check pdf format and begin extraction of results
    
    if 'W/G' not in columns:  # if older pdf format without a 'W/G column'
            
        df2 = pd.DataFrame(columns=names)        
    
        new_df = df2.iloc[:, :-2]    # remove Remarks and N/R columns
         
        for i in range(row_index+1,(len(df)-1)):                    
            
            if not re.search('^\d', table[i]): # skip row if disqualified or withdrawn (does not begin with a digit)
            
                continue
       
            string=table[i]
                                
            lpos = re.search('\s', string)
                #rpos = re.search('\sQ\s', s)
                #rpos = re.search('\s[A-Z]{2,4}', s)
            
            matches=re.findall(school_pattern, string)  # find how many uppercase shortforms for school
                                
            if len(matches)==2:
                
                rpos=re.search(matches[1], string) # choose second match if there are two 
                    
            else:
                    
                rpos = re.search(old_pattern, string)
                
            
        # Splice string to extract name of competitor

            
            l = string[:lpos.start()] # left string post splicing
            r = string[rpos.start():] # right string post slicing
         
            name=string[lpos.start():rpos.start()] # extract competitor name
                                
            list=r.split(' ')   # split the parts after the name    
            
            combined=l+ ',' + name + ', '.join(list)

            row=combined.split(",")
                        
                        
            if ('W/G' not in names) and len(row)!=6:  # if W/G not in list of columns
            
                n=len(row)-6
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
            
            elif ('W/G' in names) and len(row)>8:
            
                n=len(row)-8
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
                                        
            new_df.loc[len(new_df)] = row
            
            
        return new_df
            
    elif 'W/G' in columns: # new format 
    
            
        master_df=new_format_parser(row_index, names, table, master_df) 
     
    ##        master_df= pd.concat([temp_df, master_df])
        
#        print(master_df)
    
    return master_df

In [154]:
# To do:
# 1. code old_format_parser function for old format pdfs
# 2. Test iteration over several pages using pdf.pages[0]
# 3. Modify code to iterate over second stranded page and add data

""


In [153]:
# Test iteration over more than one page

#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/sectrack_result_01p_1-1/sectrack_result_01p_1-1.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/Deconstructed/S01-01_TO_11_deconstructed.pdf"

 
# Iterate over files in directory

#sorted_items=sort_directory(directory)

master_df=pd.DataFrame()  # initialize empty master df

splitted=None

with pdfplumber.open(file) as pdf:
    
    for i in range(len(pdf.pages)):
        
        temp=splitted
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()
        
        if i==0:
        
            splitted=text.splitlines()            
            
        else: # stitch list from second page
            
            temp=text.splitlines()
            
            splitted.extend(temp)
            
    master_df=extraction(splitted, master_df)

                


None
[[None, 'DNQ\nDNQ\nDNQ', ''], [None, None, ''], [None, None, ''], [None, None, ''], [None, None, ''], [None, None, ''], ['', None, '']]
None
None
None
None
[[''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], ['']]
None
None
None
[['', '+0 .4', None, ''], ['', '-0. 1', '', ''], ['', '+o .3', '', ''], ['', '-0. 1', '', ''], ['', '+0 .1', '', ''], ['', '+0 .1', '', ''], ['', '+o .1', '', ''], ['', '+o .3', '', ''], ['', '-0. 6', '', ''], ['', '+0 .2', '', ''], ['', '+o .1', '', ''], ['', '+0 .1', '', ''], ['', '-0. 1', '', ''], ['', '+0 .1', '', ''], ['', '+o .1', '', ''], ['', '-0. 1', '', ''], ['', '+0 .2', '', ''], ['', '+0 .3', '', ''], ['', '+o .1', '', ''], ['', '+o .4', '', ''], ['', '+0 .1', '', ''], ['', '+0 .1', '', ''], ['', '+o .3', '', ''], ['', '0. 0', '', ''], ['', '+o .4', '', ''], ['', '-0. 1', '', ''], ['', '+o .1', '', ''], ['', '+0 .1', '', ''], ['', '0. 0', '', ''], ['', '-0. 1', '', ''], ['', '+o .1', '', '']]
[['', '+0 .1', None, ''], ['', 

In [152]:
print(table)

None


In [150]:
master_df

,POS,Competitor,Q,Tag,Team,Heat,Ln,Result,Pts,W/G,Remarks,NR
0,1,OUMALEE THEEJUTHA,Q,135,HCI,hl,10,04:27.93,,,,
1,2,JONATHAN TAN,Q,214,RI,hl,8,04:30.66,,,,
2,3,JACOB TAN,Q,121,HCI,h2,3,04:30.38,,,,
3,4,BRYAN CHEUNG JIA JIE,Q,29,ACS(I),h2,6,04:31.70,,,,
4,5,PRANAV SHREEDHAR,Q,230,RI,h3,13,04:28.98,,,,
5,6,JAMES LACHLAN CHIN,Q,38,ACS(!),h3,6,04:30.81,,,,
6,7,MERVYN ONG SHAO XUAN,q,14,ACJC,h3,1,04:31.27,,,,
7,8,ASHER JIREH TAY LER EN,q,264,SJI,h2,14,04:34.00,,,,
8,9,FERRELL LEE,q,34,ACS(!),hl,3,04:35.04,,,,
9,10,LEE WEN JIE,q,152,JPJC,h3,3,04:35.96,,,,


In [56]:
# ONLY For 2024 NSG pdfs--latest version

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024PDF/sectrack_result_01p.pdf"

 
# Iterate over files in directory

#sorted_items=sort_directory(directory)

new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
#event_pattern='\:\s.{5,30}\s(\-|\()'

event_pattern='(\:\s.{5,32})\sT'

final_pattern = '.{4,30}\('


school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
year_pattern='\d\d\d\d'

div_pattern='([A-Z]\-Boys)|([A-Z]\-Girls)'

wind_pattern='(H[0-9])\s\:\s(\+\d\.\d|\-\d\.\d)'

event_mapping={'100m': 'Sprint', '100m': 'Sprint'}    


df_table=pd.DataFrame()  # initialize empty master df

df=None



with pdfplumber.open(file) as pdf:
        
        
    for i in range(len(pdf.pages)):    
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()        
        
        df=pd.DataFrame(text.splitlines())
        
                        
        for index, row in df.iterrows(): # find row containing event details and column names
                
        
            if 'Event' in row[0]:
            
                details=row[0]
                                                
                if 'BOYS' in details or 'Boys' in details:
                    gender='Male'
                
                if 'GIRLS' in details or 'Girls' in details:
                    gender='Female'    
                
                # Slice event name. Must be done in 2 stages.   
                
                
                epos = re.search(event_pattern, details)  
                                
                        
                split_end=epos.end()-1      # adjust the splicing position to capture the event + division
                split_start=epos.start()+1
    
                event=details[split_start:split_end]
                
                pos=re.search(final_pattern, event)  # new
            
                final_end=pos.end()-1      # new
                final_start=pos.start()+1  # new
                
                final_match=event[final_start:final_end] # finally extract event name
                
                
                
            # Assign high level event type
                
                if 'Throw' in final_match:
                    category_event='Throw'
                elif 'Jump' in final_match:
                    category_event='Jump'
                elif 'Vault' in final_match:
                    category_event='Jump'
                elif '100' in final_match:
                    category_event='Sprint'
                elif '1500' in final_match:
                    category_event='Mid'
                elif 'Javelin' in final_match:
                    category_event='Throw'
                elif 'Shot' in final_match:
                    category_event='Throw'
                elif '400' in final_match:
                    category_event='Sprint'
                elif '800' in final_match:
                    category_event='Mid'
                elif '400' in final_match:
                    category_event='Sprint'
                elif 'Hurdles' in final_match:
                    category_event='Hurdles'
                elif '200' in final_match:
                    category_event='Sprint'
                elif 'Walk' in final_match:
                    category_event='Walk'
                elif '3000' in final_match:
                    category_event='Long'
                elif '5000' in final_match:
                    category_event='Long'
                elif '2000' in final_match:
                    category_event='Mid'
                elif 'Discus' in final_match:
                    category_event='Throw'
                
                if 'Relay' in final_match: # Change from Sprint to Relay for relays only
                    category_event='Relay'

               
                # Slice division
        
                divpos = re.search(div_pattern, details)
                            
                s_pos=divpos.start()
                e_pos=divpos.start()+1
    
                div=details[s_pos:e_pos] # extract division        
                                      
            
            if 'Competitor' in row[0]: # extract column names
            
                row_index=index  # where the header row is
                columns=df.iloc[row_index,0]
                names=columns.split(' ')
                
            if 'Final' in row[0]:
                
                stage="Final"
                
            if 'Heat' in row[0]:
                
                stage='Heats'
                
            if 'SF' in row[0]:
                
                stage = 'SF'    
                
                               
            if 'W/Gauge' in row[0]:
                
                details=row[0]            
                
                try:
            
                    wind=re.findall(wind_pattern, details)
                                    
                    wind_dict = {}
                    
                    for key, val in wind: # create dictionary of extracted wind data
                        wind_dict.setdefault(key, val)
                
                except:
                    
                    pass
            
                    
        temp=pd.DataFrame(table)
        
                            
                
        temp.iloc[2:,:]  # drop first 2 rows

        temp['EVENT'] = final_match
        temp['GENDER'] = gender
        temp['DIVISION'] = div
        temp['STAGE'] = stage
        temp['COMPETITION'] = 'NSG'
        temp['REGION'] = 'Local'
        temp['CATEGORY_EVENT'] = category_event
        temp['test'] = 'H'+temp[5].map(str) # create a new temp field to map across wind
        temp['SOURCE'] = 'PDF'       
        temp['DATE'] = '2024'
        temp['QUALIFICATION']=''
        temp['POINTS']=''
        temp['UNIQUE_ID']=''
        temp['COUNTRY']=''
        temp['DICT_RESULTS']=''
        temp['DOB']=''
        temp['GROUP']=''
        temp['ATHLETE_ID']=''
        temp['TIMESTAMP']=''
        temp['FREE_FIELD']=''
        temp['FREE_FIELD2']=''
        temp['FREE_FIELD3']=''
        
        
        
        
        temp['WIND']=temp['test'].map(wind_dict) # map temp field across to wind data     
        
        
        temp = temp[temp[0].str.contains("~~~ End of Listing ~~~") == False]  # drop useless rows
        temp = temp[temp[0].str.contains("W/Gauge :") == False]
        temp = temp[temp[0].str.contains("Pos") == False]
     #   temp = temp[temp[7].str.contains("DNS|NM|DQ|DNF") == False] # drop rows without results
        
            
        df_table=pd.concat([df_table, temp], axis=0)
         
        


df_table.rename(
    columns={0: "RANK", 1: "TAG_ID", 2: "NAME", 3: "TEAM", 4: "RESULT", 5: "HEAT", 6: "LANE", 7: "REMARKS"},
    inplace=True,
)
        
df_table.drop('test', axis=1, inplace=True)    # drop temp columns used to map wind data   

df_table = df_table.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
                                      'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
                                      'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'])

# Add NM/DNS to results column                            
                            
mask = df_table['REMARKS'].str.contains(r'NM', na=True)
df_table.loc[mask, 'RESULT'] = 'NM'

mask = df_table['REMARKS'].str.contains(r'DNS', na=True)
df_table.loc[mask, 'RESULT'] = 'DNS'
                            
                            
                            

df_table.to_csv('sectrack_result_01p.csv', sep=',', index=False, encoding='utf-8')
        

            
                       
                       

        
            
                


In [45]:
df_table.head(50)

,RANK,TAG_ID,NAME,TEAM,RESULT,HEAT,LANE,REMARKS,EVENT,GENDER,...,DICT_RESULTS,DOB,GROUP,ATHLETE_ID,TIMESTAMP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3,RESULT,WIND
2,1,466,Zheng Menglin,NYGH,30.19,1,10,Nanyang Girls' High School,Javelin (500g),Female,...,,,,,,,,,NM,NaN
3,2,455,"Liang Zhi Xuan, Jodie",NYGH,29.32,1,8,Nanyang Girls' High School,Javelin (500g),Female,...,,,,,,,,,NM,NaN
4,3,567,Megan Koh Jeng Peck (Xu Yingbi),SNG,28.51,1,12,CHIJ St. Nicholas Girls',Javelin (500g),Female,...,,,,,,,,,NM,NaN
5,4,549,Charissa Ong Huey Ern,SNG,25.79,1,4,CHIJ St. Nicholas Girls',Javelin (500g),Female,...,,,,,,,,,NM,NaN
6,5,242,Liam Raejia,CG,23.59,1,2,Cedar Girls' Sec School,Javelin (500g),Female,...,,,,,,,,,NM,NaN
7,6,425,He Leyu,NJC,23.06,1,3,National Junior College,Javelin (500g),Female,...,,,,,,,,,NM,NaN
8,7,452,Jordan Chang Tong,NYGH,22.57,1,16,Nanyang Girls' High School,Javelin (500g),Female,...,,,,,,,,,NM,NaN
9,8,252,Quek Xin En Isabelle,CG,22.16,1,18,Cedar Girls' Sec School,Javelin (500g),Female,...,,,,,,,,,NM,NaN
10,9,436,Thirumalai Nivedita,NJC,21.47,1,5,National Junior College,Javelin (500g),Female,...,,,,,,,,,NM,NaN
11,10,257,Shannon Tan Ying Jia,CG,21.38,1,7,Cedar Girls' Sec School,Javelin (500g),Female,...,,,,,,,,,NM,NaN


In [54]:
df_table

,RANK,TAG_ID,NAME,TEAM,SEED,RESULT,QUALIFICATION,HEAT,LANE,WIND,...,DOB,GROUP,CATEGORY_EVENT,ATHLETE_ID,SOURCE,REMARKS,TIMESTAMP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
2,1,466,Zheng Menglin,NYGH,NaN,30.19,,1,10,NaN,...,,,Throw,,PDF,Nanyang Girls' High School,,,,
3,2,455,"Liang Zhi Xuan, Jodie",NYGH,NaN,29.32,,1,8,NaN,...,,,Throw,,PDF,Nanyang Girls' High School,,,,
4,3,567,Megan Koh Jeng Peck (Xu Yingbi),SNG,NaN,28.51,,1,12,NaN,...,,,Throw,,PDF,CHIJ St. Nicholas Girls',,,,
5,4,549,Charissa Ong Huey Ern,SNG,NaN,25.79,,1,4,NaN,...,,,Throw,,PDF,CHIJ St. Nicholas Girls',,,,
6,5,242,Liam Raejia,CG,NaN,23.59,,1,2,NaN,...,,,Throw,,PDF,Cedar Girls' Sec School,,,,
7,6,425,He Leyu,NJC,NaN,23.06,,1,3,NaN,...,,,Throw,,PDF,National Junior College,,,,
8,7,452,Jordan Chang Tong,NYGH,NaN,22.57,,1,16,NaN,...,,,Throw,,PDF,Nanyang Girls' High School,,,,
9,8,252,Quek Xin En Isabelle,CG,NaN,22.16,,1,18,NaN,...,,,Throw,,PDF,Cedar Girls' Sec School,,,,
10,9,436,Thirumalai Nivedita,NJC,NaN,21.47,,1,5,NaN,...,,,Throw,,PDF,National Junior College,,,,
11,10,257,Shannon Tan Ying Jia,CG,NaN,21.38,,1,7,NaN,...,,,Throw,,PDF,Cedar Girls' Sec School,,,,


In [33]:
# Open all files in directory

def sort_directory(directory):
    items = os.listdir(directory)
    sorted_items = sorted(items)
    return sorted_items


directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/"
 
# Iterate over files in directory

sorted_items=sort_directory(directory)

sorted_items.pop(0)

master_df=pd.DataFrame()  # initialize empty master df

for file in sorted_items:
    
    print(file)
    
    temp = pd.read_csv(file)
    
    master_df=pd.concat([master_df, temp], axis=0)
    
    


sectrack_result_01p.csv
sectrack_result_02p.csv
sectrack_result_03r.csv
sectrack_result_04p.csv
sectrack_result_05q.csv
sectrack_result_06p.csv
sectrack_result_07p.csv
sectrack_result_08p.csv
sectrack_result_09r.csv
sectrack_result_10p.csv
sectrack_result_11.csv
sectrack_result_12.csv
sectrack_result_13.csv
sectrack_result_14.csv
sectrack_result_15.csv
sectrack_result_16a.csv
sectrack_result_17a.csv
sectrack_result_18.csv
sectrack_result_19.csv
sectrack_result_20.csv
sectrack_result_21b.csv
sectrack_result_22a.csv


In [34]:
master_df

,RANK,TAG_ID,NAME,TEAM,SEED,RESULT,QUALIFICATION,HEAT,LANE,WIND,...,DOB,GROUP,CATEGORY_EVENT,ATHLETE_ID,SOURCE,REMARKS,TIMESTAMP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,1.0,466.0,Zheng Menglin,NYGH,NaN,30.19,NaN,1.0,10.0,NaN,...,NaN,NaN,Throw,NaN,PDF,Nanyang Girls' High School,NaN,NaN,NaN,NaN
1,2.0,455.0,"Liang Zhi Xuan, Jodie",NYGH,NaN,29.32,NaN,1.0,8.0,NaN,...,NaN,NaN,Throw,NaN,PDF,Nanyang Girls' High School,NaN,NaN,NaN,NaN
2,3.0,567.0,Megan Koh Jeng Peck (Xu Yingbi),SNG,NaN,28.51,NaN,1.0,12.0,NaN,...,NaN,NaN,Throw,NaN,PDF,CHIJ St. Nicholas Girls',NaN,NaN,NaN,NaN
3,4.0,549.0,Charissa Ong Huey Ern,SNG,NaN,25.79,NaN,1.0,4.0,NaN,...,NaN,NaN,Throw,NaN,PDF,CHIJ St. Nicholas Girls',NaN,NaN,NaN,NaN
4,5.0,242.0,Liam Raejia,CG,NaN,23.59,NaN,1.0,2.0,NaN,...,NaN,NaN,Throw,NaN,PDF,Cedar Girls' Sec School,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,5.0,NaN,St. Andrew's Junior College,SAJC,NaN,03:34.65,NaN,1.0,3.0,NaN,...,NaN,NaN,Relay,NaN,PDF,St. Andrew's Junior College,NaN,NaN,NaN,NaN
214,6.0,NaN,Yishun Innova Junior College,YIJC,NaN,03:38.01,NaN,1.0,7.0,NaN,...,NaN,NaN,Relay,NaN,PDF,Yishun Innova Junior,NaN,NaN,NaN,NaN
215,7.0,NaN,St. Joseph's Institution,SJI,NaN,03:38.06,NaN,1.0,8.0,NaN,...,NaN,NaN,Relay,NaN,PDF,St. Joseph's Institution,NaN,NaN,NaN,NaN
216,8.0,NaN,National Junior College,NJC,NaN,03:40.57,NaN,1.0,9.0,NaN,...,NaN,NaN,Relay,NaN,PDF,National Junior College,NaN,NaN,NaN,NaN


In [35]:
master_df.to_csv('2024_ALL.csv', sep=',', index=False, encoding='utf-8')


In [26]:
splitted.extend(temp)

In [27]:
print(splitted)

['Audited List', 'Event: S13-03 4 X 400m Relay - B DIVISION GIRLS Heats', 'Schools National Record Singapore Sports School SSP (ET)04:01.98s 2011', 'Championships Record Singapore Sports School SSP (ET)04:01.98s 2011', 'POS Competitor Q Tag Team Heat Ln Result Pts W/G Remarks NR', '1 SINGAPORE SPORTS SCHOOL Q 720 SSP h3 3 04:23.41', "2 NANYANG GIRLS' HIGH Q 1022 NYGH h3 6 04:24.02", 'SCHOOL', "3 CRESCENT GIRLS' SCHOOL Q 791 CGS h3 2 04:24.09", "4 CEDAR GIRLS' SECONDARY Q 871 CG h2 4 04:25.16", 'SCHOOL', "5 CHIJ ST. NICHOLAS GIRLS' Q 656 SNG h2 5 04:30.99", 'SCHOOL (SECONDARY)', '6 NATIONAL JUNIOR COLLEGE Q 810 NJC h2 8 04:32.69', '7 DUNMAN HIGH SCHOOL Q 930 DHS h2 2 04:37.82', '8 CHIJ SECONDARY (TOA PAYOH) Q 747 HIJ h3 5 04:39.52', "9 RAFFLES GIRLS' SCHOOL Q 845 RGS h2 1 04:40.60", '(SECONDARY)', '10 AHMAD IBRAHIM SECONDARY 648 AIS h1 6 04:43.48', 'SCHOOL', '11 PAYA LEBAR METHODIST 953 PLMGS h3 1 04:48.28', "GIRLS' SCHOOL (SECONDARY)", '12 NORTH VISTA SECONDARY 707 NV h1 2 04:49.01', '

### Do not modify below code

In [83]:
def extraction(table, master_df):  # latest version to process 2023. takes stitchesd list from multiple paged pdfs

    global index, row_index
        
        
    # Regex patterns to extract competitor names, event name etc.
    
    #new_pattern='\d\s.{5,20}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='\d\s[A-Z\s\,]{5,30}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='^[A-Z\s]*$'
    
    new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'
    
    event = None
    
    names = None
    
    temp_df = pd.DataFrame()

    
    # Put extracted info from pdf into a df and extract event information
    
    df=pd.DataFrame(table)
    
    columns=[]
    
        
    for index, row in df.iterrows(): # find row containing event details and column names
                
        
        if 'Event' in row[0]:
            
            event=row[0]
                                    
            epos = re.search(event_pattern, event)
                        
            split_end=epos.end()-1      # adjust the splicing position to only capture the event name
            split_start=epos.start()+1

            event=event[split_start:split_end]
            
            print(event)
                                    
            
            if 'BOYS' in event or 'Boys' in event:
                gender='Male'
                
            if 'GIRLS' in event or 'Girls' in event:
                gender='Female'    
                
            
            if 'Finals' in event:
                round='Finals'
            
            region='Local'          
                  
            
        if 'Competitor' in row[0]: # extract column names
            
            row_index=index
            columns=df.iloc[row_index,0]
            names=columns.split(' ')
                        
    
#    if (event and names) is None:  # if it is a stranded page (no header) that is cutoff from the main pdf
                                # Change this code to add iteration over pages. Just append splitted 2nd page.
                                
#            temp_df = master_df.iloc[:0]  # initiate empty df and copy column names
                                    
#            row_index=0   # if no header 
            
#            names = None  # retrieve master df and append instead
            
#            print('here')
            
#            print(table)
                                            
#            temp_df=new_format_parser(row_index, names, table, master_df) # need to concat master_df and temp df
            
#            print('reached this 3rd point for a stranded pdf')
            
#            master_df= pd.concat([master_df, temp_df])  # this concat is not working as temp_df is empty
                    
#            return master_df # exit function

            
    
    # Check pdf format and begin extraction of results
    
    if 'W/G' not in columns:  # if older pdf format without a 'W/G column'
            
        df2 = pd.DataFrame(columns=names)        
    
        new_df = df2.iloc[:, :-2]    # remove Remarks and N/R columns
         
        for i in range(row_index+1,(len(df)-1)):                    
            
            if not re.search('^\d', table[i]): # skip row if disqualified or withdrawn (does not begin with a digit)
            
                continue
       
            string=table[i]
                                
            lpos = re.search('\s', string)
                #rpos = re.search('\sQ\s', s)
                #rpos = re.search('\s[A-Z]{2,4}', s)
            
            matches=re.findall(school_pattern, string)  # find how many uppercase shortforms for school
                                
            if len(matches)==2:
                
                rpos=re.search(matches[1], string) # choose second match if there are two 
                    
            else:
                    
                rpos = re.search(old_pattern, string)
                
            
        # Splice string to extract name of competitor

            
            l = string[:lpos.start()] # left string post splicing
            r = string[rpos.start():] # right string post slicing
         
            name=string[lpos.start():rpos.start()] # extract competitor name
                                
            list=r.split(' ')   # split the parts after the name    
            
            combined=l+ ',' + name + ', '.join(list)

            row=combined.split(",")
                        
                        
            if ('W/G' not in names) and len(row)!=6:  # if W/G not in list of columns
            
                n=len(row)-6
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
            
            elif ('W/G' in names) and len(row)>8:
            
                n=len(row)-8
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
                                        
            new_df.loc[len(new_df)] = row
            
            
        return new_df
            
    elif 'W/G' in columns: # new format     
            
        master_df=new_format_parser(row_index, names, table, master_df) 
        
    ##        master_df= pd.concat([temp_df, master_df])
        
#        print(master_df)
    
    return master_df

In [441]:
# Newest version to ingest 2023 PDFs
# Almost complete except for some formatting effects

def new_format_parser(row_index, names, table, master_df):
    
                
    # Define regex patterns
    
    new_pattern='\d\s.{5,40}?\s(Qfy|Q\s|q\s|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'
    
    stranded = "no"  # flag for stranded pdf pages
    
    ex_pattern = 'NM|DNS'
    
    tag_pattern = '\d\d|\d\d\d'
    
    gap_pattern = ':\s\d'
        
    # Put table that has been processed into a dataframe
    
    df=pd.DataFrame(table)
                
    columns=[]
    
    list=[]
                
    
    # Initialize new df and remove unnecessary columns
    
    if row_index!=0 and names is not None: #check that its coming from not a stranded pdf with no header
            
        df2 = pd.DataFrame(columns=names) 
        
    
        #new_df = df2.iloc[:, :-2]    # remove 'Remarks' and 'NR' columns
        
        new_df = df2    # retain 'REMARKS' and 'NR' columns
    
    else:
        
        print('reached this first point for a stranded pdf')
        
        stranded="yes"
        
        new_df = master_df.iloc[:0] # new_df columns to come from the master df
        
        row_index=-1  # move back row_index for a stranded pdf
        
    
#    for i in range(row_index+1,(len(df)-1)):  # iterate over df - only stranded pdf should start from zero. Previous code - revert to original  


#  check for stranded pdfs with stranded names

    for i in range(row_index+1,(len(df))):  # iterate over df - only stranded pdf should start from index zero    
        
                                            
        if not re.search('^\d', table[i]) and not re.search(ex_pattern, table[i]): # look for stranded parts of names (does not begin with a digit). But need to test for standed names in stranded pdf
                
            try:  # remove try block to revert back to previous working state
                                                
                row[1]=row[1]+table[i] # retrieve previous row and add back stranded string to name. Stranded pdf will throw and error here. [1] refers to index position 1 corresponding to name
                                                                                
                new_df.drop(new_df.tail(1).index,inplace=True) # drop last row with incomplete name
                                
                pad_value = ' '
                pad_size = 12 - len(row)

                padded = [*row, *[pad_value] * pad_size]  # add extra columns to make it 12
            
                                                                        
                new_df.loc[len(new_df)] = padded  # add back amended row
                
                list.append(padded)
                                            
                                                                    
                continue  # then skip to next row
            
            except:
                
                print('reached this second point for a stranded pdf')
                
                pass                 # move on if it is actually a stranded pdf
                              
        if not re.search('^\d', table[i]) and re.search(ex_pattern, table[i]): # look for legitimate names that don't contain a rank e.g. NM
        
            print('past', table[i])
                            
            string=table[i] 
            
            tag_pos=re.search(tag_pattern, string)
            
            name = string[:tag_pos.start()]
            
            everything_else = string[tag_pos.start():]
            
            list=everything_else.split(' ')
                        
            list.insert(0, name)
            
            list.insert(0, ' ') # insert space for rank
            
            list.insert(2, ' ') # insert space for Q/q         
            
            list.insert(7, ' ')
            
            list.insert(7, ' ')
            
            list.insert(7, ' ')
            
            list.extend([' '])
                                                    
            new_df.loc[len(new_df)] = list
            
        if re.search('^\d', table[i]) and re.search(ex_pattern, table[i]):  # DNS pattern that begins with a digit
            
            string=table[i] 
            
            matches=re.findall(tag_pattern, string)  # find second set of digits corresponding to  tag ID
                                
            if len(matches)==2:
                
                rpos=re.search(matches[1], string) # choose second match if there are two 
            
            
            element = string[:rpos.start()]  # contains rank and name together
            
            rank = element[0:3]  # extract rank only
                        
            name =element[3:]  # extract name only
                 
            everything_else = string[rpos.start():]
            
            list=everything_else.split(' ')
            
            print(list)
            
            list.insert(0, name)
                                    
            list.insert(0, rank)
            
            list.insert(2, ' ') # insert space for Q/q         
            
            list.insert(7, ' ')
                        
            list.insert(7, ' ')
            
            list.extend([' '])
            
            print('DNS', list)
            
            
            new_df.loc[len(new_df)] = list

            
        if table[i].isdigit():  # exit function if it's not a proper line (e.g. number fragments with no content)  
            
            return new_df
        
        
        if i==len(df):     # exit if end of df is reached
            
            return new_df
       
        
        
        # Else split lines per normal format
    
        if not re.search(ex_pattern, table[i]):  # skip those lines with NM/DNS   
        
            string=table[i] 
                    
            try:
            
                string=string.replace(": ", ":")       #delete gap in timing result
                        
            except:
            
                pass
                    
        
            lpos = re.search('\s', string)
            rpos = re.search(new_pattern, string)
            
                                                        
        
        # Splice string to extract name of competitor

            if 'Qfy' not in string:
        
                split_pos_end=rpos.end()-1      # adjust the splicing position to only capture the name
                split_pos_start=lpos.start()+1

            
                left_string = string[:lpos.start()] # left string post splicing
                right_string = string[split_pos_end:] # right string post slicing
                         
                name=string[split_pos_start:split_pos_end]  # extract competitor name
                
            elif 'Qfy' in string:
                
                split_pos_end=rpos.end()-3      # adjust the splicing position to only capture the name
                split_pos_start=lpos.start()+1

            
                left_string = string[:lpos.start()] # left string post splicing
                right_string = string[split_pos_end:] # right string post slicing
                         
                name=string[split_pos_start:split_pos_end]  # extract competitor name
                
                    
                               
            list=right_string.split(' ')        # put everything coming after the name into a list  
                                                        
                
 #       if len(list)==6:    # check length of list. If less than 7, append blank fields for Pos and W/G 
                    
#            list.extend([' ', ' ', ' ', ' '])
                    
#        elif len(list)==5:
                    
#            list.extend([' ', ' ', ' ', ' '])
                                                            
                
            combined=left_string+ ';' + name +'; '+ '; '.join(list)  # use ';' in case names have a ','

            row=combined.split(';')   
                                                                                                 

        # check if Q or q between tag and name. If not, fill in with empty space.
        # don't overwrite master df if there is a stranded pdf
                
            if row[2].strip()=='Q' or row[2].strip()=='q' or row[2].strip()=='Qfy' or row[2] is None :
            
                pad_value = ' '
                pad_size = 12 - len(row)

                final_list = [*row, *[pad_value] * pad_size]  # add extra columns to make it 12
            
                print('HERE1', final_list)
                                
                new_df.loc[len(new_df)] = final_list


            else:
                
                row.insert(2, ' ')
            
                pad_value = ' '
                pad_size = 12 - len(row)

                final_list = [*row, *[pad_value] * pad_size]  # add extra columns to make it 12
            
                print('HERE2', final_list)
            
                list.append(final_list)
            
                            
                new_df.loc[len(new_df)] = final_list
                                
            
            
            
            
# QAed to this point for a stranded pdf. Need to figure out how to attach stranded row to master df
            
            
#            master_df=new_df      # previous   
    
    return new_df

In [446]:
# Test iteration over more than one page

#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/sectrack_result_01p_1-1/sectrack_result_01p_1-1.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2023/S01-01_TO_11_deconstructed_1-1/S01-01_TO_11_deconstructed_22-25.pdf"

 
# Iterate over files in directory

#sorted_items=sort_directory(directory)

temp_df=pd.DataFrame()  # initialize empty temp df

master_df=pd.DataFrame()

splitted=None

with pdfplumber.open(file) as pdf:
    
    for i in range(len(pdf.pages)):
        
        temp=splitted
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()
                
        if i==0:
        
            splitted=text.splitlines()
            
            temp_df=extraction(splitted, temp_df)
            
        else: # stitch list from second page
            
            temp=text.splitlines()
            
            splitted.extend(temp)
            
            temp_df=extraction(splitted, temp_df)
                        
        master_df=pd.concat([master_df, temp_df], axis=0)


            
    #master_df=extraction(splitted, master_df)

                


 S0l-10 100m 
HERE2 ['1', 'TAN YING TONG, SHANNON Q', ' ', ' ', ' 81', ' CG', ' hl', ' 8', ' 00:13.75', ' +0.1', ' ', ' ']
HERE2 ['2', 'ALEXA TAN LIYI Q', ' ', ' ', ' 259', ' MGS', ' h3', ' 2', ' 00:13.87', ' 0.0', ' ', ' ']
HERE2 ['3', "HEATHER TAN RAE'QI Q", ' ', ' ', ' 53', ' CG', ' h3', ' 5', ' 00:13.88', ' 0.0', ' f', ' ']
HERE2 ['4', 'TEH XIN NI CHARLOTTE (ZHENG Q', ' ', ' ', ' 159', ' SNG', ' h8', ' 7', ' 00:13.90', ' +1.4', ' ', ' ']
HERE2 ['5', 'KRISHA SHASHINI D/O JAI RAJ Q', ' ', ' ', ' 376', ' SSP', ' h12', ' 6', ' 00:13.91', ' +1.3', ' ', ' ']
HERE2 ['6', 'KAYLEY SEAH XIN EN Q', ' ', ' ', ' 59', ' CG', ' h4', ' 5', ' 00:13.98', ' +0.3', ' ', ' ']
HERE2 ['7', 'CHONG KATE LYNN, LEA Q', ' ', ' ', ' 276', ' NYGH', ' hll', ' 7', ' 00:14.09', ' +1.5', ' ', ' ']
HERE2 ['8', 'NG REI ANN Q', ' ', ' ', ' 217', ' DHS', ' hl', ' 2', ' 00:14.16', ' +0.1', ' ', ' ']
HERE2 ['9', 'HUANG TIANA! Q', ' ', ' ', ' 283', ' NYGH', ' h7', ' 6', ' 00:14.45', ' +1.3', ' ', ' ']
HERE2 ['10', 'HANNAH

ValueError: cannot set a row with mismatched columns

In [445]:
master_df

,POS,Competitor,Q,Tag,Team,Heat,Ln,Result,Pts,W/G,Remarks,NR
0,1,TEH YING SHAN Q,,,528,CCHY,h8,8,00:12.94,+0.7,,
1,2,CHLOE CHEE EN-YA Q,,,309,MGS,h6,1,00:13.17,-0.5,,
2,3,JAYME NG SOK CHEE Q,,,156,SNG,h9,2,00:13.25,+0.4,,
3,4,MEGAN ANNE YING KA MUN Q,,,76,CG,h9,1,00:13.27,+0.4,,
4,5,"SOH HWEI EN, RACHEL (SU QHUI'EN RACHEL)",,,358,NYGH,h3,1,00:13.30,-0.3,,
...,...,...,...,...,...,...,...,...,...,...,...,...
48,49,ARYANNA ZAHARA BINTE MOHAMED RAFI,,102,KC,hl0,7,00:15.44,-1.0,,,
49,50,NG JING XUAN ( HUANG JINGXUAN),,129,HIJ,hl,8,00:15.48,-0.3,,,
50,51,TEO YI JIE,,33,BV,h3,4,00:15.49,-0.3,,,
51,52,BELLELYN ONG,,11,AIS,h6,4,00:15.49,-0.5,,,


In [400]:
pattern='\d\s.{5,40}?\s(Q\s|q\s|\d)'

string='2 GUOK XUE QIAN 143 SNG Finals 6 8.91 7'


In [401]:
re.search(pattern, string)

<re.Match object; span=(0, 17), match='2 GUOK XUE QIAN 1'>